# TSA Chapter 5: Bitcoin ARCH Effects

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/TSA/blob/main/TSA_ch5/TSA_ch5_btc_arch/TSA_ch5_btc_arch.ipynb)

Evidence of ARCH effects in Bitcoin: squared returns and their ACF.

In [ ]:
!pip install numpy pandas matplotlib scipy yfinance arch statsmodels -q

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from scipy import stats
# Chart style
BLUE = '#1A3A6E'; RED = '#DC3545'; GREEN = '#2E7D32'
ORANGE = '#E67E22'; GRAY = '#666666'; PURPLE = '#8E44AD'

plt.rcParams.update({
    'figure.facecolor': 'none', 'axes.facecolor': 'none',
    'savefig.facecolor': 'none', 'savefig.transparent': True,
    'axes.grid': False, 'font.size': 10, 'axes.labelsize': 11,
    'axes.titlesize': 12, 'legend.fontsize': 9, 'xtick.labelsize': 9,
    'ytick.labelsize': 9, 'axes.spines.top': False, 'axes.spines.right': False,
    'lines.linewidth': 1.0, 'axes.linewidth': 0.6,
    'legend.facecolor': 'none', 'legend.framealpha': 0, 'legend.edgecolor': 'none',
    'figure.dpi': 150,
})

def save_chart(fig, name):
    fig.savefig(f'{name}.pdf', bbox_inches='tight', transparent=True, dpi=150)
    fig.savefig(f'{name}.png', bbox_inches='tight', transparent=True, dpi=150)
    print(f'Saved: {name}')

def legend_bottom(ax, ncol=2, y=-0.18):
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, y), ncol=ncol, frameon=False)
import yfinance as yf
from statsmodels.tsa.stattools import acf

In [ ]:
import yfinance as yf

# Download Bitcoin data
btc = yf.download('BTC-USD', start='2019-01-01', end='2025-12-31', progress=False)
if isinstance(btc.columns, pd.MultiIndex):
    btc.columns = btc.columns.get_level_values(0)
btc['Return'] = btc['Close'].pct_change() * 100
btc.dropna(inplace=True)
btc_returns = btc['Return']
print(f"Bitcoin: {len(btc)} observations")

In [ ]:
# BTC squared returns
sq = btc_returns**2
fig, ax = plt.subplots(figsize=(10, 3))
ax.fill_between(sq.index, 0, sq.values, color=ORANGE, alpha=0.6,
                label='$r_t^2$ (squared returns)')
ax.set_title('Bitcoin Squared Returns (Volatility Proxy)', fontweight='bold')
ax.set_xlabel('Date'); ax.set_ylabel('Return$^2$ (%$^2$)')
ax.set_ylim(0, 400)
ax.xaxis.set_major_locator(mdates.YearLocator())
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
legend_bottom(ax, ncol=1, y=-0.18)
save_chart(fig, 'btc_squared_returns'); plt.show()

In [ ]:
# ACF of squared returns
acf_vals, confint = acf(sq.dropna(), nlags=40, alpha=0.05)
lags = np.arange(len(acf_vals))

fig, ax = plt.subplots(figsize=(8, 3))
ax.bar(lags[1:], acf_vals[1:], width=0.6, color=BLUE, label='ACF of $r_t^2$')
ci_upper = confint[1:, 1] - acf_vals[1:]
ax.fill_between(lags[1:], -ci_upper, ci_upper, color=BLUE, alpha=0.15, label='95% CI')
ax.axhline(0, color='black', lw=0.5)
ax.set_xlabel('Lag'); ax.set_ylabel('ACF')
ax.set_title('ACF of Squared BTC Returns', fontweight='bold')
legend_bottom(ax, ncol=2, y=-0.22)
save_chart(fig, 'btc_acf_squared'); plt.show()